In [1]:
from neo4j import GraphDatabase

class Neo4j:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

neo4j = Neo4j("bolt://localhost:7687", "neo4j", "password")

In [47]:
# neo4j.driver.execute_query("MATCH (p) RETURN p LIMIT 2")

def execute(query):
    with neo4j.driver.session() as session:
        result = session.run(query)
        return result

# Import Data


In [59]:
clean_up = "MATCH (n) DETACH DELETE n"
show_all = "MATCH (n) RETURN n"

# Import files: /Users/enioohmaye/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-41d35f05-e4e0-4f5a-a04e-64b41bb80839/import

# Load SPINs
import_students = """
        LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
        WITH row WHERE row.FirstName IS NOT NULL
        MERGE (s:Student {firstName:row.FirstName, lastName:row.LastName})
"""

import_choices = """
        LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
        WITH row WHERE row.{0} IS NOT NULL
        MERGE (s:SPIN {{SPIN:row.{0}}})
"""

import_student_WANTS_spins = """
        LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row
        WITH row WHERE row.FirstName IS NOT NULL
        MATCH (stud:Student) WHERE row.FirstName = stud.firstName and row.LastName = stud.lastName
        MATCH (spin:SPIN) WHERE spin.SPIN = row.{0}
        MERGE (stud)-[wants:WANTS {{ranking:{1}}}]->(spin)
"""


import_student_level = """
        LOAD CSV WITH HEADERS FROM 'file:///2_Weekly_Student_Class.csv' AS row
        WITH row WHERE row.`First Name` IS NOT NULL
        MATCH (stud:Student) WHERE row.`First Name` = stud.firstName and row.`Last Name` = stud.lastName
        SET stud.level = row.Level
"""
import_rooms = """
        LOAD CSV WITH HEADERS FROM 'file:///Rooms.csv' AS row
        WITH row WHERE row.Room IS NOT NULL
        MERGE (r:Room {name:row.Room, capacity:row.Capacity, type:row.Type})
"""

In [46]:
format(import_choices.format('Choice'))

"LOAD CSV WITH HEADERS FROM 'file:///SPINChoice.csv' AS row\n        WITH row WHERE row.Choice IS NOT NULL\n        MERGE (s:SPIN {SPIN:row.Choice})\n"

In [60]:
execute(clean_up)
execute(show_all)
execute(import_students)
execute(import_choices.format("Choice01"))
execute(import_choices.format("Choice02"))
execute(import_choices.format("Choice03"))
execute(import_choices.format("Choice04"))
execute(import_choices.format("Choice05"))
execute(import_choices.format("IntensiveChoice01"))
execute(import_choices.format("IntensiveChoice02"))
execute(import_choices.format("IntensiveChoice03"))
execute(import_student_WANTS_spins.format("Choice01", 10))
execute(import_student_WANTS_spins.format("Choice02", 8))
execute(import_student_WANTS_spins.format("Choice03", 6))
execute(import_student_WANTS_spins.format("Choice04", 4))
execute(import_student_WANTS_spins.format("Choice05", 2))
execute(import_student_WANTS_spins.format("IntensiveChoice01", 10))
execute(import_student_WANTS_spins.format("IntensiveChoice02", 8))
execute(import_student_WANTS_spins.format("IntensiveChoice03", 6))
execute (import_student_level)

# Queries


In [67]:
# Top choices sorted by number of students choosing it as Choice01
q = """
    MATCH (stud:Student)-[r:WANTS]->(spin)
    WITH spin, r.ranking as rank, stud.level as level 
    RETURN count(*) as Total, level, rank, spin ORDER BY rank DESC,Total DESC
"""
records, summary, keys = neo4j.driver.execute_query(q)
for record in records:
    print(record.data())

{'Total': 22, 'level': 'A1-1', 'rank': 10, 'spin': {'SPIN': '*Learning Japanese w/ Games and Board Games(A1-B2)'}}
{'Total': 17, 'level': None, 'rank': 10, 'spin': {'SPIN': '*Learning Japanese w/ Games and Board Games(A1-B2)'}}
{'Total': 15, 'level': 'A1-1', 'rank': 10, 'spin': {'SPIN': 'リスニング&アニメ (Listening and Anime)'}}
{'Total': 14, 'level': 'A1-1', 'rank': 10, 'spin': {'SPIN': '*Communication(A1-B2)'}}
{'Total': 14, 'level': None, 'rank': 10, 'spin': {'SPIN': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）'}}
{'Total': 12, 'level': None, 'rank': 10, 'spin': {'SPIN': '*Communication(A1-B2)'}}
{'Total': 10, 'level': None, 'rank': 10, 'spin': {'SPIN': 'ことば（Vocabulary ）'}}
{'Total': 10, 'level': None, 'rank': 10, 'spin': {'SPIN': 'リスニング&アニメ (Listening and Anime)'}}
{'Total': 10, 'level': 'A1-1', 'rank': 10, 'spin': {'SPIN': 'N5 漢字 Kanji for JLPT N5（A1.1-A1.2）'}}
{'Total': 10, 'level': 'A1-1', 'rank': 10, 'spin': {'SPIN': 'A1 Grammar (A1.2-A1.3)'}}
{'Total': 9, 'level': 'A1-1', 'rank': 10, 'spin': 